In [1]:
import mwclient
import time

site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1195811696),
             ('parentid', 1195731806),
             ('user', 'Aquillion'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=1, tm_mday=15, tm_hour=11, tm_min=1, tm_sec=22, tm_wday=0, tm_yday=15, tm_isdst=-1)),
             ('comment', 'adding another source')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [16]:
!pip install transformers


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
!pip install --upgrade ipywidgets jupyter


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install torch torchvision torchaudio


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
find_sentiment("I hate you")

-0.9991129040718079

In [9]:
find_sentiment("I hate you")

-0.9991129040718079

In [6]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline(text[:250])[0]  # Analyze the first 250 characters of the input text
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1  # If the sentiment is negative, return the negative of the score
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
find_sentiment("I hate you")

-0.9991129040718079

In [12]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline(text[:250])[0]  # Analyze the first 250 characters of the input text
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -score  # If the sentiment is negative, return the negative of the score
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [13]:
find_sentiment("It is dump")

-0.9995519424807107

In [8]:
find_sentiment("I love you")

0.9998656511306763

In [13]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])

    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    comment = rev.get("comment", "")  # Get the comment if it exists, otherwise use an empty string
    if comment:  # Only analyze sentiment if there is a comment
        sentiment = find_sentiment(comment)
        edits[date]["sentiments"].append(sentiment)


In [21]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["mean_sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"]if s<0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["mean_sentiment"] = 0
        edits[key]["neg_sentiment"] = 0

    del edits[key]["sentiments"]

In [22]:
edits

{'2009-03-08': {'edit_count': 4,
  'mean_sentiment': -0.9834070404370626,
  'neg_sentiment': 1.0},
 '2009-08-05': {'edit_count': 1, 'mean_sentiment': 0, 'neg_sentiment': 0},
 '2009-08-06': {'edit_count': 2,
  'mean_sentiment': 0.9957457184791565,
  'neg_sentiment': 0.0},
 '2009-08-14': {'edit_count': 1,
  'mean_sentiment': 0.9300208687782288,
  'neg_sentiment': 0.0},
 '2009-10-13': {'edit_count': 2,
  'mean_sentiment': -0.2275010645389557,
  'neg_sentiment': 0.5},
 '2009-11-18': {'edit_count': 1,
  'mean_sentiment': 0.883950412273407,
  'neg_sentiment': 0.0},
 '2009-12-08': {'edit_count': 1,
  'mean_sentiment': -0.9869275689125061,
  'neg_sentiment': 1.0},
 '2009-12-17': {'edit_count': 1,
  'mean_sentiment': -0.9975171089172363,
  'neg_sentiment': 1.0},
 '2010-02-23': {'edit_count': 1,
  'mean_sentiment': -0.9994946718215942,
  'neg_sentiment': 1.0},
 '2010-03-18': {'edit_count': 1,
  'mean_sentiment': 0.8758774995803833,
  'neg_sentiment': 0.0},
 '2010-04-13': {'edit_count': 4,
  'mea

In [23]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [24]:
edits_df

,edit_count,mean_sentiment,neg_sentiment
2009-03-08,4,-0.983407,1.000000
2009-08-05,1,0.000000,0.000000
2009-08-06,2,0.995746,0.000000
2009-08-14,1,0.930021,0.000000
2009-10-13,2,-0.227501,0.500000
...,...,...,...
2024-01-08,2,-0.997788,1.000000
2024-01-10,1,0.876924,0.000000
2024-01-11,1,0.924098,0.000000
2024-01-12,3,0.315349,0.333333


In [26]:
edits_df.index = pd.to_datetime(edits_df.index)

In [29]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08", end=datetime.today())

In [30]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11',
               '2024-01-12', '2024-01-13', '2024-01-14', '2024-01-15',
               '2024-01-16', '2024-01-17'],
              dtype='datetime64[ns]', length=5429, freq='D')

In [31]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [32]:
edits_df

,edit_count,mean_sentiment,neg_sentiment
2009-03-08,4,-0.983407,1.0
2009-03-09,0,0.000000,0.0
2009-03-10,0,0.000000,0.0
2009-03-11,0,0.000000,0.0
2009-03-12,0,0.000000,0.0
...,...,...,...
2024-01-13,0,0.000000,0.0
2024-01-14,0,0.000000,0.0
2024-01-15,2,0.080183,0.5
2024-01-16,0,0.000000,0.0


In [33]:
rolling_edits = edits_df.rolling(30).mean()

In [34]:
rolling_edits

,edit_count,mean_sentiment,neg_sentiment
2009-03-08,NaN,NaN,NaN
2009-03-09,NaN,NaN,NaN
2009-03-10,NaN,NaN,NaN
2009-03-11,NaN,NaN,NaN
2009-03-12,NaN,NaN,NaN
...,...,...,...
2024-01-13,0.433333,-0.053455,0.177778
2024-01-14,0.433333,-0.053455,0.177778
2024-01-15,0.500000,-0.050782,0.194444
2024-01-16,0.500000,-0.050782,0.194444


In [35]:
rolling_edits=rolling_edits.dropna()

In [36]:
rolling_edits

,edit_count,mean_sentiment,neg_sentiment
2009-04-06,0.133333,-0.032780,0.033333
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-01-13,0.433333,-0.053455,0.177778
2024-01-14,0.433333,-0.053455,0.177778
2024-01-15,0.500000,-0.050782,0.194444
2024-01-16,0.500000,-0.050782,0.194444


In [37]:
rolling_edits.to_csv("wikipedia_edits.csv")